<a href="https://colab.research.google.com/github/johanjun/Google_Machine_Learning_Bootcamp_KR_2021/blob/main/kaggle/RSNA_MICCAI_BRAIN_TUMOR/rsna_miccai_brain_tumor_johanjun_1001_overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pydicom

     |████████████████████████████████| 2.0 MB 5.1 MB/s 


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm
import random
import pydicom
import cv2
import pickle

import matplotlib.pyplot as plt
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
data_dir = '/content/drive/MyDrive/Google_Machine_Learning_Bootcamp_KR_2021/RSNA_MICCAI/data/rsna-miccai/'

In [5]:
train_df = pd.read_csv(data_dir + "train_labels.csv")
test_df = pd.read_csv(data_dir + "sample_submission.csv")

In this discussion a competition host has notified that there are some issues with these 3 cases
Patient IDs -

1. 00109 (FLAIR images are blank)
2. 00123 (T1w images are blank)
3. 00709 (FLAIR images are blank)
<br><br>Hence these can be excluded

In [6]:
#refer: https://www.kaggle.com/arnabs007/part-1-rsna-miccai-btrc-understanding-the-data
EXCLUDE = [109, 123, 709]
train_df = train_df[~train_df.BraTS21ID.isin(EXCLUDE)]

In [7]:
TYPES = ["FLAIR", "T1w", "T1wCE", "T2w"] #mpMRI scans

In [8]:
def load_dicom(path, size = 64): #load DICOM files
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array #returns a numpy.ndarray containing the pixel data
    if np.max(data) != 0:
        data = data / np.max(data) #standardizes so that the pixel values are between 0 and 1
    data = (data * 255).astype(np.uint8) #rescales to 0 and 255
    return cv2.resize(data, (size, size))

In [9]:
def get_all_image_paths(BraTS21ID, image_type, folder="train"): #get an array of all the images of a particular type or a particular patient id
    assert(image_type in TYPES) #only in types
    patient_path = os.path.join(data_dir + "%s/" % folder, str(BraTS21ID).zfill(5)) #다른 폴더일 수도 있음
    #print(lambda x: int(x[:-4].split("-")[-1]))
    
    paths = sorted(glob.glob(os.path.join(patient_path, image_type, "*")), key=lambda x: int(x[:-4].split("-")[-1])) #sort
    #print(paths)
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)
    if num_images < 10:
        jump = 1
    else:
        jump = 3
        
    return np.array(paths[start:end:jump])

In [10]:
def get_all_images(BraTS21ID, image_type, folder="train", size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(BraTS21ID, image_type, folder)]

In [11]:
def get_all_data_train(image_type):
    global train_df
    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        tmp_x = train_df.loc[i]
        images = get_all_images(int(tmp_x["BraTS21ID"]), image_type, "train", IMAGE_SIZE)
        label = tmp_x["MGMT_value"]

        X += images
        y += [label] * len(images)
        train_ids += [int(tmp_x["BraTS21ID"])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

def get_all_data_test(image_type):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        tmp_x = test_df.loc[i]
        images = get_all_images(int(tmp_x["BraTS21ID"]), image_type, "test", IMAGE_SIZE)
        X += images
        test_ids += [int(tmp_x["BraTS21ID"])] * len(images)

    return np.array(X), np.array(test_ids)

In [12]:
#pickling
with open(data_dir+'X_T1wCE.pkl', 'rb') as f:
  X_T1wCE = pickle.load(f)

with open(data_dir+'y_T1wCE.pkl', 'rb') as f:
  y_T1wCE = pickle.load(f)

with open(data_dir+'train_idt_T1wCE.pkl', 'rb') as f:
  train_idt_T1wCE = pickle.load(f)

with open(data_dir+'X_test_T1wCE.pkl', 'rb') as f:
  X_test_T1wCE = pickle.load(f)

with open(data_dir+'test_idt_T1wCE.pkl', 'rb') as f:
  test_idt_T1wCE = pickle.load(f)
#pickling
with open(data_dir+'X_FLAIR.pkl', 'rb') as f:
  X_FLAIR = pickle.load(f)

with open(data_dir+'y_FLAIR.pkl', 'rb') as f:
  y_FLAIR = pickle.load(f)

with open(data_dir+'train_idt_FLAIR.pkl', 'rb') as f:
  train_idt_FLAIR = pickle.load(f)

with open(data_dir+'X_test_FLAIR.pkl', 'rb') as f:
  X_test_FLAIR = pickle.load(f)

with open(data_dir+'test_idt_FLAIR.pkl', 'rb') as f:
  test_idt_FLAIR = pickle.load(f)
#pickling
with open(data_dir+'X_T2w.pkl', 'rb') as f:
  X_T2w = pickle.load(f)

with open(data_dir+'y_T2w.pkl', 'rb') as f:
  y_T2w = pickle.load(f)

with open(data_dir+'train_idt_T2w.pkl', 'rb') as f:
  train_idt_T2w = pickle.load(f)

with open(data_dir+'X_test_T2w.pkl', 'rb') as f:
  X_test_T2w = pickle.load(f)

with open(data_dir+'test_idt_T2w.pkl', 'rb') as f:
  test_idt_T2w = pickle.load(f)
#pickling
with open(data_dir+'X_T1w.pkl', 'rb') as f:
  X_T1w = pickle.load(f)

with open(data_dir+'y_T1w.pkl', 'rb') as f:
  y_T1w = pickle.load(f)

with open(data_dir+'train_idt_T1w.pkl', 'rb') as f:
  train_idt_T1w = pickle.load(f)

with open(data_dir+'X_test_T1w.pkl', 'rb') as f:
  X_test_T1w = pickle.load(f)

with open(data_dir+'test_idt_T1w.pkl', 'rb') as f:
  test_idt_T1w = pickle.load(f)

In [28]:
# %load train_lib.py

def train():
    import wandb
    config_defaults = {
        'MRI': 'FLAIR',
        'test_size' :  0.05,
        'N_EPOCHS' : 20,
        'N_BATCH' : 32,

    }
    wandb.init(project='RSNA_1001_DEEPCNN',
               config=config_defaults,
               magic=True)

    config = wandb.config
    
    MRI_type = config.MRI

    if MRI_type == 'FLAIR':
      X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X_FLAIR, y_FLAIR, train_idt_FLAIR, test_size=config.test_size)
    elif MRI_type == 'T1wCE':
      X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X_T1wCE, y_T1wCE, train_idt_T1wCE, test_size=config.test_size)
    elif MRI_type == 'T1w':
      X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X_T1w, y_T1w, train_idt_T1w, test_size=config.test_size)
    elif MRI_type == 'T2w':
      X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X_T2w, y_T2w, train_idt_T2w, test_size=config.test_size)

    y_train = to_categorical(y_train)
    y_valid = to_categorical(y_valid)

    X_train = np.expand_dims(X_train, axis=-1)
    X_valid = np.expand_dims(X_valid, axis=-1)

    IMAGE_SIZE = 64
    N_EPOCHS = config.N_EPOCHS
    N_BATCH = config.N_BATCH
    N_TRAIN = X_train.shape[0]
    N_VAL = X_valid.shape[0]
    steps_per_epoch = N_TRAIN / N_BATCH
    validation_steps = int(np.ceil(N_VAL / N_BATCH))

    def create_model():
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='SAME', 
                                    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      
      model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      model.add(tf.keras.layers.Dropout(0.1))

      model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      model.add(tf.keras.layers.Dropout(0.1))

      model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=5, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      model.add(tf.keras.layers.Dropout(0.1))

      model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=4, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Flatten())
      model.add(tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.Dropout(0.4))
      model.add(tf.keras.layers.Dense(2, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
      return model
    
    model = create_model()
    

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.AUC()])
    
    model.fit(
    x=X_train,
    y=y_train,
    epochs=N_EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_valid, y_valid))

In [29]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'MRI': {
            'values': ['FLAIR', 'T1wCE', 'T1w', 'T2w']
        },
        'N_EPOCHS': {
            'values': [20, 100]
        },
        'N_BATCH': {
            'values': [32, 128]
        },
        'test_size': {
            'values': [0.05, 0.2]
        },
    }
}

In [16]:
%pip install -q wandb

     |████████████████████████████████| 1.7 MB 5.0 MB/s 
     |████████████████████████████████| 139 kB 60.3 MB/s 
     |████████████████████████████████| 180 kB 49.6 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


In [17]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [31]:
import wandb
sweep_id = wandb.sweep(sweep_config, project='RSNA_1001_DEEPCNN')

Create sweep with ID: uo1eh4wc
Sweep URL: https://wandb.ai/johanjunme/RSNA_1001_DEEPCNN/sweeps/uo1eh4wc


In [32]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 3g87y2fz with config:
wandb: 	MRI: FLAIR
wandb: 	N_BATCH: 32
wandb: 	N_EPOCHS: 20
wandb: 	test_size: 0.05
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/20
369/369 [==============================] - 8s 17ms/step - loss: 2.8020 - auc: 0.5534 - val_loss: 1.4239 - val_auc: 0.4962
Epoch 2/20
369/369 [==============================] - 5s 14ms/step - loss: 1.0764 - auc: 0.5928 - val_loss: 0.8536 - val_auc: 0.5520
Epoch 3/20
369/369 [==============================] - 5s 14ms/step - loss: 0.8175 - auc: 0.6179 - val_loss: 0.7595 - val_auc: 0.5985
Epoch 4/20
369/369 [==============================] - 5s 14ms/step - loss: 0.7471 - auc: 0.6445 - val_loss: 0.7445 - val_auc: 0.6594
Epoch 5/20
369/369 [==============================] - 5s 14ms/step - loss: 0.7349 - auc: 0.6495 - val_loss: 0.7141 - val_auc: 0.6500
Epoch 6/20
369/369 [==============================] - 5s 14ms/step - loss: 0.7159 - auc: 0.6694 - val_loss: 0.6963 - val_auc: 0.6889
Epoch 7/20
369/369 [==============================] - 5s 14ms/step - loss: 0.6925 - auc: 0.7105 - val_loss: 0.8610 - val_auc: 0.5265
Epoch 8/20
369/369 [==============================] - 5s 14ms/step - 

auc,▁▂▂▃▃▃▄▄▄▅▆▆▇▇▇█████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_auc,▁▂▃▄▄▄▂▅▆▆▇▅▇▇▇█▇███
val_loss,█▄▃▃▃▃▄▂▂▂▂▄▁▂▁▁▂▁▂▂
auc,0.97507
best_epoch,15
best_val_loss,0.48277
epoch,19
loss,0.23292
val_auc,0.86977


wandb: Agent Starting Run: 82getqtq with config:
wandb: 	MRI: FLAIR
wandb: 	N_BATCH: 32
wandb: 	N_EPOCHS: 20
wandb: 	test_size: 0.2
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/20
311/311 [==============================] - 7s 18ms/step - loss: 3.0089 - auc: 0.5579 - val_loss: 1.5326 - val_auc: 0.5868
Epoch 2/20
311/311 [==============================] - 5s 15ms/step - loss: 1.1957 - auc: 0.5903 - val_loss: 0.8964 - val_auc: 0.6392
Epoch 3/20
311/311 [==============================] - 5s 15ms/step - loss: 0.8511 - auc: 0.6052 - val_loss: 0.7400 - val_auc: 0.6516
Epoch 4/20
311/311 [==============================] - 5s 15ms/step - loss: 0.7717 - auc: 0.6200 - val_loss: 0.8689 - val_auc: 0.6631
Epoch 5/20
311/311 [==============================] - 5s 15ms/step - loss: 0.7379 - auc: 0.6429 - val_loss: 0.7444 - val_auc: 0.6362
Epoch 6/20
311/311 [==============================] - 5s 15ms/step - loss: 0.7276 - auc: 0.6620 - val_loss: 0.7619 - val_auc: 0.6683
Epoch 7/20
311/311 [==============================] - 5s 15ms/step - loss: 0.7295 - auc: 0.6709 - val_loss: 0.7498 - val_auc: 0.6575
Epoch 8/20
311/311 [==============================] - 5s 15ms/step - 

...

auc,▁▄▆▇▇███████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc,▁▃▅▆█▅▇██▇█▇██▇████████████████████▇████
val_loss,█▃▂▂▁▃▂▁▁▁▁▂▁▁▂▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
auc,0.99936
best_epoch,13
best_val_loss,0.25026
epoch,99
loss,0.02432
val_auc,0.97717


wandb: Agent Starting Run: yckjbrkt with config:
wandb: 	MRI: T2w
wandb: 	N_BATCH: 128
wandb: 	N_EPOCHS: 100
wandb: 	test_size: 0.2
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/100
105/105 [==============================] - 7s 44ms/step - loss: 3.6778 - auc: 0.5757 - val_loss: 2.3715 - val_auc: 0.6443
Epoch 2/100
105/105 [==============================] - 3s 31ms/step - loss: 1.9090 - auc: 0.6389 - val_loss: 1.4117 - val_auc: 0.6684
Epoch 3/100
105/105 [==============================] - 3s 30ms/step - loss: 1.2072 - auc: 0.6942 - val_loss: 1.0102 - val_auc: 0.6575
Epoch 4/100
105/105 [==============================] - 3s 30ms/step - loss: 0.8973 - auc: 0.7387 - val_loss: 0.7968 - val_auc: 0.7592
Epoch 5/100
105/105 [==============================] - 3s 30ms/step - loss: 0.6766 - auc: 0.8157 - val_loss: 0.8198 - val_auc: 0.7297
Epoch 6/100
105/105 [==============================] - 3s 31ms/step - loss: 0.5906 - auc: 0.8381 - val_loss: 0.5977 - val_auc: 0.8042
Epoch 7/100
105/105 [==============================] - 3s 30ms/step - loss: 0.5016 - auc: 0.8792 - val_loss: 0.5013 - val_auc: 0.8607
Epoch 8/100
105/105 [==============================] - 3s 30ms

auc,▁▃▅▇▇▇▇█████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc,▁▁▅▆▆▆▆▆▇███▇████████▇▇█████████▇▇███▇██
val_loss,█▃▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▂▁▁▂▂▁▁
auc,0.99835
best_epoch,24
best_val_loss,0.28903
epoch,99
loss,0.06603
val_auc,0.94975


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
